In [0]:
# Change directory to VSCode workspace root so that relative path loads work correctly. Turn this addition off with the DataScience.changeDirOnImportExport setting
import os
try:
	os.chdir(os.path.join(os.getcwd(), '..'))
	print(os.getcwd())
except:
	pass


In [0]:
import os
try:
    os.chdir(os.path.join(os.getcwd(), 'course_berkeley/homework/hw2'))
    print(os.getcwd())
except:
    pass

 # Problem 1
 We want to show that
 \begin{equation}
 \sum_{t'=1}^{T}\mathbb{E}_{\tau \sim p_{\theta}(\tau)}[\nabla_{\theta}log \pi_{\theta}(a_{t'}|s_{t'})(b(s_{t'}))]=0
 \end{equation}
 \begin{align}
 \sum_{t=1}^{T}\mathbb{E}_{\tau \sim p_{\theta}(\tau)}[\nabla_{\theta}log \pi_{\theta}(a_t|s_t)(b(s_t))]
 &=\sum_{t=1}^{T}\int p_{\theta}(\tau)\nabla_{\theta}log \pi_{\theta}(a_t|s_t)(b(s_t))d\tau\\
 %&=\sum_{t=1}^{T}\int p_{\theta}(s_t,a_t)p_{\theta}(\tau / s_t,a_t\mid s_t,a_t)\nabla_{\theta}log \pi_{\theta}(a_t|s_t)(b(s_t))d\tau\\
 &=\sum_{t=1}^{T}\int \int p_{\theta}(s_t,a_t)(\int p_{\theta}(\tau / s_t,a_t\mid s_t,a_t)\nabla_{\theta}log \pi_{\theta}(a_t|s_t)(b(s_t))d\tau )d s_t da_t)\\
 &=\sum_{t=1}^{T}\int \int p_{\theta}(s_t,a_t)\nabla_{\theta}log \pi_{\theta}(a_t|s_t)b(s_t)\int p_{\theta}(\tau / s_t,a_t\mid s_t,a_t)d\tau d s_t d a_t\\
 &=\sum_{t=1}^{T}\int \int p_{\theta}(s_t,a_t)\nabla_{\theta}log \pi_{\theta}(a_t|s_t)b(s_t) d s_t d a_t\\
 &=\sum_{t=1}^{T}\int \int p(s_{t+1}|a_t,s_t)\pi_{\theta}(a_t|s_t)\nabla_{\theta}log \pi_{\theta}(a_t|s_t)b(s_t) d s_t d a_t\\
 &=\sum_{t=1}^{T}\int \int p(s_{t+1}|a_t,s_t)b(s_t)\nabla_{\theta}\pi_{\theta}(a_t|s_t) d s_t d a_t\\
 &=\sum_{t=1}^{T} \nabla_{\theta} \int \int p(s_{t+1}|a_t,s_t)b(s_t)\pi_{\theta}(a_t|s_t) d s_t d a_t \\
 &=\sum_{t=1}^{T} \nabla_{\theta} \int \int p_{\theta}(a_t,s_t)b(s_t) d s_t d a_t \\
 \end{align}
Where $\nabla_{\theta}log \pi_{\theta}(A_t|S_t)b(S_t)=b(S_t)\nabla_{\theta}log \pi_{\theta}(A_t|S_t)$, since $b$ is assumed not to depend on $\theta$ (in practice it could be estimated using a different neural network with parameters $\phi$).

In [ ]:
  &=\mathbb{E}_{(A_t,S_t)}[\mathbb{E}_{\tau / (A_t,S_t) \sim p_{\theta}(\tau / S_t,A_t\mid S_t,A_t)}[0\mid A_t , S_t]]\\
  &= \mathbf{0}
 Where $\nabla_{\theta}log \pi_{\theta}(A_t|S_t)(b(S_t)) = 0$ given $A_t$ and $S_t$. Therefore 

In [0]:
from train_pg_f18 import train_PG
seed = 1
train_PG(
                exp_name="sb_no_rtg_dna",
                env_name="CartPole-v0",
                n_iter=100,
                gamma=0.99,
                min_timesteps_per_batch=1000,
				learning_rate=0.001,
				reward_to_go=None,
                max_path_length=99999999999,
                animate=False,
                logdir= None , #os.path.join("data",'%d'%seed),
				normalize_advantages=False,
                nn_baseline=None, 
                seed=seed,
                n_layers=2,
                size=100
                )

In [1]:
%load_ext autoreload

%autoreload 2

## Test continous  actions
We use the logprob expression
```
sy_logprob_n =  -1/2 * tf.reduce_sum(inverse_variance *(diff*diff),axis=1)
```
Even though
```
sy_logprob_n = - 1/2*(tf.log(variance)  + tf.reduce_sum(inverse_variance *(diff*diff),axis=1))
```
would have been more correct as to follow the logprob. For this small example this makes the variance be maximized after, and thus gives more incorrect results, however for the real examples it performes the same, probably due to maximization errors when doing the other thing. It is also what is suggested in Levines slides.

In [1]:
from train_pg_f18 import train_PG, Agent
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
computation_graph_args = {
    'n_layers': 1,
    'ob_dim': 3,
    'ac_dim': 1,
    'discrete': False,
    'size': 2,
    'learning_rate': 0.005,
}

sample_trajectory_args = {
    'animate': False,
    'max_path_length': 1000,
    'min_timesteps_per_batch': 1000,
}

estimate_return_args = {
    'gamma': 0.99,
    'reward_to_go': True,
    'nn_baseline': False,
    'normalize_advantages': False,
}

with tf.Graph().as_default() as graph:
    agent = Agent(computation_graph_args, sample_trajectory_args, estimate_return_args)
    agent.build_computation_graph()
    tf_config = tf.ConfigProto(inter_op_parallelism_threads=1,
        intra_op_parallelism_threads=1,
        gpu_options = tf.GPUOptions(
            #per_process_gpu_memory_fraction=1./16. # 1gb
            allow_growth=True
        )
    )
    with tf.Session(config=tf_config).as_default() as sess:
        

        sess.__enter__() # equivalent to `with self.sess:`
        tf.global_variables_initializer().run() #pylint: disable=E1101


        # check that the loss is computed correct
        print(sess.run([agent.policy_parameters, agent.sy_logprob_n, agent.loss], {
            agent.sy_ob_no: [[0.5,0.5,0.5],
                            [0.5,0.5,0.5],
                            [0.5,0.5,0.5]],
            agent.sy_ac_na: [[2.0], [2.0], [2.0]],
            agent.sy_adv_n: [1,1,1]
        
        })
        )
        # check that the actions look correct
        n_samples = 100
        sampled_actions = sess.run(agent.sy_sampled_ac, {
            agent.sy_ob_no: [[0.5,0.5,0.5]]*n_samples,
            #agent.sy_ac_na: [[2.0]]*n_samples,
            #agent.sy_adv_n: [1]*n_samples
        
        })
        figure =  plt.figure()
        plot = plt.hist(sampled_actions)
        figure.show()
        print("std: {}, mean: {}".format(np.std(sampled_actions),np.mean(sampled_actions)))
        n_training = 4000
        for i in range(n_training):
            sy_ac_na = np.random.normal(loc=5.0 , scale=0.1,size=n_samples)
            sess.run(agent.update_op, {
                agent.sy_ob_no: [[0.5,0.5,0.5]]*n_samples,
                agent.sy_ac_na: sy_ac_na.reshape(-1,1),
                agent.sy_adv_n: [1]*n_samples

            })
            sampled_actions = sess.run(agent.sy_sampled_ac, {
                agent.sy_ob_no: [[0.5,0.5,0.5]]*n_samples,
                #agent.sy_ac_na: [[2.0]]*n_samples,
                #agent.sy_adv_n: [1]*n_samples
            })
            if i %100 == 99:
                print("it: {}, std: {}, mean: {}".format(i, np.std(sampled_actions),np.mean(sampled_actions)))
        
        figure =  plt.figure()
        plot = plt.hist(sampled_actions)
        figure.show()

[(array([[0.],
       [0.],
       [0.]], dtype=float32), array([0.], dtype=float32)), array([-2., -2., -2.], dtype=float32), 2.0]
std: 0.9857619404792786, mean: 0.053708259016275406


/zhome/30/0/70339/.local/lib/python3.6/site-packages/matplotlib/figure.py:445: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  % get_backend())


it: 99, std: 4.66080379486084, mean: 5.440565586090088
it: 199, std: 6.097592830657959, mean: 4.187251091003418
it: 299, std: 44.20418167114258, mean: 9.711440086364746
it: 399, std: 111.15580749511719, mean: -4.385540962219238
it: 499, std: 151.47842407226562, mean: 27.827970504760742
it: 599, std: 186.75601196289062, mean: -14.619833946228027
it: 699, std: 214.3723907470703, mean: 23.425809860229492
it: 799, std: 224.56509399414062, mean: -21.883249282836914
it: 899, std: 266.2421875, mean: -15.566503524780273
it: 999, std: 271.6304626464844, mean: 19.704458236694336
it: 1099, std: 304.3190002441406, mean: 26.475332260131836
it: 1199, std: 300.62518310546875, mean: 10.948735237121582
it: 1299, std: 336.2844543457031, mean: 23.704082489013672
it: 1399, std: 375.9537353515625, mean: 68.61624145507812
it: 1499, std: 345.3172302246094, mean: 39.9020881652832
it: 1599, std: 362.7534484863281, mean: 26.962749481201172
it: 1699, std: 387.88238525390625, mean: -39.475982666015625
it: 1799, s

## Test baseline

In [23]:
from train_pg_f18 import train_PG, Agent
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
computation_graph_args = {
    'n_layers': 1,
    'ob_dim': 3,
    'ac_dim': 1,
    'discrete': False,
    'size': 2,
    'learning_rate': 0.005,
}

sample_trajectory_args = {
    'animate': False,
    'max_path_length': 1000,
    'min_timesteps_per_batch': 1000,
}

estimate_return_args = {
    'gamma': 0.99,
    'reward_to_go': True,
    'nn_baseline': True,
    'normalize_advantages': False,
}

with tf.Graph().as_default() as graph:
    agent = Agent(computation_graph_args, sample_trajectory_args, estimate_return_args)
    agent.build_computation_graph()
    tf_config = tf.ConfigProto(inter_op_parallelism_threads=1,
        intra_op_parallelism_threads=1,
        gpu_options = tf.GPUOptions(
            #per_process_gpu_memory_fraction=1./16. # 1gb
            allow_growth=True
        )
    )
    with tf.Session(config=tf_config).as_default() as sess:
        

        sess.__enter__() # equivalent to `with sess:`
        tf.global_variables_initializer().run() #pylint: disable=E1101

        # check that the actions look correct
        n_samples = 1
        ob_no =  [[0.0,0.0,0.0],[-1.,-1.,-1.]]*n_samples
        q_n = [1,-1]*n_samples
        value = sess.run(agent.baseline_prediction, {
                agent.sy_ob_no: ob_no
            })
        print("Value of state before {}".format(value))
        target_n = (q_n - np.mean(q_n))/np.std(q_n)
        print("Target values {}".format(target_n))
        n_training = 1000
        print("Number of training baseline: {}".format(n_training))
        for _ in range(n_training):
            sess.run(agent.baseline_update_op, {
                agent.sy_ob_no: ob_no,
                agent.sy_target_n: target_n
            })
            value = sess.run(agent.baseline_prediction, {
                    agent.sy_ob_no: ob_no
                })
        print("Value of state after {}".format(value))

Value of state before [0. 0.]
Target values [ 1. -1.]
Number of training baseline: 1000
Value of state after [ 0.9999992 -0.9999991]
